In [0]:
%sql
SELECT * FROM shm.contract.flat
WHERE text = ''

In [0]:
-- Step 1: Parse documents and store parsed output
CREATE OR REPLACE TEMP VIEW parsed_pdf_docs AS
SELECT
  path,
  ai_parse_document(
    content,
    map(
      'imageOutputPath', '/Volumes/shm/contract/images',
      'descriptionElementTypes', '*'
    )
  ) AS parsed
FROM shm.contract.bytes
WHERE path IN (
  SELECT path 
  FROM shm.contract.flat f
  WHERE f.text = ''
);

-- Step 2: Explode pages and create final table
CREATE OR REPLACE TABLE pdf_page_images AS
SELECT
  path,
  page:id AS id,
  page:image_uri AS image_uri
FROM parsed_pdf_docs
LATERAL VIEW
  explode(cast(parsed:document.pages as array<variant>)) p AS page;

In [0]:
%sql

SELECT AI_QUERY(
  'databricks-claude-sonnet-4',
  'Extract all the text from this document and return it as a single string', 
  files => content
  ) AS output FROM shm.contract.bytes
WHERE path IN (
  SELECT path 
  FROM shm.contract.flat f
  WHERE f.text = ''
)